In [1]:
import pickle
import os

import open3d as o3d
import numpy as np

from open3d.web_visualizer import draw
from mmdetection3d import data, demo, configs, checkpoints

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


# KITTI

## Load train infos

In [2]:
train_infos_path = os.path.join(
    data.__path__[0], 'datasets', 'kitti', 'kitti_infos_train.pkl')
print(train_infos_path)
train_infos = pickle.load(open(train_infos_path, 'rb'))

/home/javier/prueba/mmdetection3d/data/datasets/kitti/kitti_infos_train.pkl


In [3]:
train_infos.keys()

dict_keys(['metainfo', 'data_list'])

In [4]:
train_infos['data_list'][2]

{'sample_idx': 7,
 'images': {'CAM0': {'cam2img': [[721.5377, 0.0, 609.5593, 0.0],
    [0.0, 721.5377, 172.854, 0.0],
    [0.0, 0.0, 1.0, 0.0],
    [0.0, 0.0, 0.0, 1.0]],
   'lidar2img': [[609.6953812723476,
     -721.4215942962135,
     -1.2512579994207245,
     -167.8990963799692],
    [180.384193781635,
     7.64479865145192,
     -719.6515015339527,
     -101.23306821726784],
    [0.999945342540741,
     0.00012436549877747893,
     0.010451302863657475,
     -0.2721328139305115],
    [0.0, 0.0, 0.0, 1.0]]},
  'CAM1': {'cam2img': [[721.5377, 0.0, 609.5593, -387.5744],
    [0.0, 721.5377, 172.854, 0.0],
    [0.0, 0.0, 1.0, 0.0],
    [0.0, 0.0, 0.0, 1.0]],
   'lidar2img': [[609.6953812723476,
     -721.4215942962135,
     -1.2512579994207245,
     -555.4734963799692],
    [180.384193781635,
     7.64479865145192,
     -719.6515015339527,
     -101.23306821726784],
    [0.999945342540741,
     0.00012436549877747893,
     0.010451302863657475,
     -0.2721328139305115],
    [0.0, 0.0,

## Inference

In [5]:
from mmdet3d.apis import inference_detector, init_model
# from sensus.utils.data_converter import pc2pc_object
from mmdet3d.utils import register_all_modules

In [6]:
import numpy as np
from mmdet3d.structures.points import get_points_type
from mmdet3d.datasets.transforms.loading import LoadPointsFromFile

def pc2pc_object(pc, pipeline):
    pipeline_dict = {**pipeline[0]}
    assert pipeline_dict['type'] == 'LoadPointsFromFile'
    pipeline_dict.pop('type')
    pc_loader = LoadPointsFromFile(**pipeline_dict)

    pc = pc.reshape(-1, pc_loader.load_dim)
    pc = pc[:, pc_loader.use_dim]
    if pc_loader.norm_intensity:
            assert len(pc_loader.use_dim) >= 4, \
                f'When using intensity norm, expect used dimensions >= 4, got {len(pc_loader.use_dim)}'  # noqa: E501
            pc[:, 3] = np.tanh(pc[:, 3])
    attribute_dims = None

    if pc_loader.shift_height:
        floor_height = np.percentile(pc[:, 2], 0.99)
        height = pc[:, 2] - floor_height
        pc = np.concatenate(
            [pc[:, :3],
                np.expand_dims(height, 1), pc[:, 3:]], 1)
        attribute_dims = dict(height=3)

    if pc_loader.use_color:
        assert len(pc_loader.use_dim) >= 6
        if attribute_dims is None:
            attribute_dims = dict()
        attribute_dims.update(
            dict(color=[
                pc.shape[1] - 3,
                pc.shape[1] - 2,
                pc.shape[1] - 1,
            ]))

    points_class = get_points_type(pc_loader.coord_type)
    pc_object = points_class(
        pc, points_dim=pc.shape[-1], attribute_dims=attribute_dims)

    return pc_object, pc

In [7]:
register_all_modules()


# build the model from a config file and a checkpoint file
model_cfg = os.path.join(configs.__path__[0],
    'second/second_hv_secfpn_8xb6-80e_kitti-3d-3class.py')
checkpoint_path = os.path.join(checkpoints.__path__[0],
    'hv_second_secfpn_6x8_80e_kitti-3d-3class_20210831_022017-ae782e87.pth')
pcd_path = os.path.join(demo.__path__[0],
    'data/kitti/000008.bin')
device = 'cuda:0'
model = init_model(model_cfg, checkpoint_path, device=device)

/home/javier/prueba/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:92: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


Loads checkpoint by local backend from path: /home/javier/prueba/mmdetection3d/checkpoints/hv_second_secfpn_6x8_80e_kitti-3d-3class_20210831_022017-ae782e87.pth


In [8]:
# Running model with numpy point cloud gives error when later running with pcd_path
model = init_model(model_cfg, checkpoint_path, device=device)
result, data = inference_detector(model, pcd_path)

Loads checkpoint by local backend from path: /home/javier/prueba/mmdetection3d/checkpoints/hv_second_secfpn_6x8_80e_kitti-3d-3class_20210831_022017-ae782e87.pth


/home/javier/miniconda3/envs/openmmlab/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552411/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
with open(pcd_path, 'rb') as f:
    points = np.fromfile(f, dtype=np.float32, count=-1)

In [15]:
pc_object, pc = pc2pc_object(points, model.cfg.test_pipeline)
result, data = inference_detector(model, pc)

In [11]:
print(result.pred_instances_3d.labels_3d)
print(result.pred_instances_3d.scores_3d)
print(result.pred_instances_3d.bboxes_3d.tensor.shape)

tensor([0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([0.1304, 0.1172, 0.3625, 0.3570, 0.9088, 0.8917, 0.8498, 0.7471, 0.7245,
        0.6413, 0.6306, 0.5602, 0.4315, 0.3938], device='cuda:0')
torch.Size([14, 7])


In [16]:
# Same result after processing points (maybe processing under the hood when
# using np.array pc)
points = points.reshape(-1, 4)
result, data = inference_detector(model, points)

In [13]:
print(result.pred_instances_3d.labels_3d)
print(result.pred_instances_3d.scores_3d)
print(result.pred_instances_3d.bboxes_3d.tensor.shape)

tensor([0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([0.1304, 0.1172, 0.3625, 0.3570, 0.9088, 0.8917, 0.8498, 0.7471, 0.7245,
        0.6413, 0.6306, 0.5602, 0.4315, 0.3938], device='cuda:0')
torch.Size([14, 7])


# Visualization

In [11]:
bin_path = os.path.join(demo.__path__[0],
    'data/kitti/000008.bin')
pc_path = os.path.join(
    '/home/messi/alvaro/sensus-loci/mmdetection3d/demo/kitti_pointpillars/kitti_000008',
    'kitti_000008_points.obj')
bboxes_path = os.path.join(
    '/home/messi/alvaro/sensus-loci/mmdetection3d/demo/kitti_pointpillars/kitti_000008',
    'kitti_000008_pred.obj')

In [17]:
# Read pc from bin file
with open(bin_path, 'rb') as f:
    points = np.fromfile(f, dtype=np.float32, count=-1).reshape([-1, 4])

pcd_bin = o3d.geometry.PointCloud()
pcd_bin.points = o3d.utility.Vector3dVector(points[:, :3])
print(pcd_bin)

NameError: name 'bin_path' is not defined

In [5]:
# Read pc from obj file (reading with o3d.io.read_triangle_mesh or 
# read_point_cloud does not work)
pc = []
with open(pc_path, 'rb') as f:
    for each in f.readlines():
        p1, p2, p3 = each.decode('utf-8').split(' ')[1:]
        pc.append([float(p1), float(p2), float(p3.replace('\n', ''))])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.array(pc))
print(pcd)

bboxes = o3d.io.read_triangle_mesh(bboxes_path)
bboxes.compute_vertex_normals()     # For solid rendering with lighting
bboxes_lines = o3d.geometry.LineSet().create_from_triangle_mesh(bboxes)
bboxes_lines.paint_uniform_color([1, 0, 0])
print(bboxes_lines)

PointCloud with 16897 points.
LineSet with 360 lines.


In [29]:
draw([pcd, bboxes_lines], width=900, height=600, point_size=2)

WebVisualizer(window_uid='window_11')

In [13]:
draw([pcd_bin, bboxes_lines], width=900, height=600, point_size=2)

WebVisualizer(window_uid='window_0')